# Calculate global mean surface temperature from LongRunMIP models
#### Christopher Callahan
#### Christopher.W.Callahan.GR@dartmouth.edu

Cosine latitude weighted global mean of tas

#### Mechanics
Dependencies

In [ ]:
import xarray as xr
import numpy as np
import sys
import os
import datetime
import pandas as pd

Model names list

In [ ]:
modelnames_fig = ['CCSM3 abrupt 2x','CCSM3 abrupt 4x','CCSM3 abrupt 8x', \
    'CESM1.0.4 abrupt 2x','CESM1.0.4 abrupt 4x','CESM1.0.4 abrupt 8x', 'CNRM-CM6.1 abrupt4x', \
    'GFDL-CM3 1pct 2x','GFDL-ESM2M 1pct 2x','GISS-E2-R 1pct 4x', \
    'GISS-E2-R abrupt 4x','HadCM3L abrupt 2x','HadCM3L abrupt 4x', \
    'HadCM3L abrupt 6x','HadCM3L abrupt 8x','IPSL-CM5A-LR abrupt 4x', \
    'MIROC3.2 1pct 2x','MIROC3.2 1pct 4x','MPIESM-1.2 abrupt 2x', \
    'MPIESM-1.2 abrupt 4x','MPIESM-1.2 abrupt 8x']

modelnames_file = ['CCSM3_abrupt2x','CCSM3_abrupt4x','CCSM3_abrupt8x', \
    'CESM104_abrupt2x','CESM104_abrupt4x','CESM104_abrupt8x', \
    'CNRMCM61_abrupt4x','GFDLCM3_1pct2x','GFDLESM2M_1pct2x','GISSE2R_1pct4x', \
    'GISSE2R_abrupt4x','HadCM3L_abrupt2x','HadCM3L_abrupt4x', \
    'HadCM3L_abrupt6x','HadCM3L_abrupt8x','IPSLCM5A_abrupt4x', \
    'MIROC32_1pct2x','MIROC32_1pct4x','MPIESM12_abrupt2x', \
    'MPIESM12_abrupt4x','MPIESM12_abrupt8x']

Data locations

In [ ]:
loc_tas = "~/" # change if using raw data
loc_out = "../Data/GMST/"

Function for listing files

In [ ]:
def list_files(directory, extension):
    
    # Credit to: http://www.martinbroadhurst.com/listing-all-files-in-a-directory-with-a-certain-extension-in-python.html
    # Requires os.listdir
    
    return (f for f in os.listdir(directory) if f.endswith(extension))

#### Analysis

In [ ]:
for i in np.arange(0,len(modelnames_file),1):
    
    model, exp = modelnames_file[i].split("_")
    print(modelnames_file[i])
    
    if modelnames_file[i] == "MPIESM11_abrupt4x":
        fname_exp_start = "tas_ann_"+modelnames_file[i]
        fname_control_start = "tas_ann_"+model+"_control"
    else:
        fname_exp_start = "tas_mon_"+modelnames_file[i]
        fname_control_start = "tas_mon_"+model+"_control"
    
    fname_exp = [f for f in os.listdir(loc_tas) if f.startswith(fname_exp_start)][0]
    fname_control = [f for f in os.listdir(loc_tas) if f.startswith(fname_control_start)][0]
    
    n1, n2, n3, n4, year_exp_nc = fname_exp.split("_")
    n1, n2, n3, n4, year_control_nc = fname_control.split("_")
    year_exp, nc = year_exp_nc.split(".")
    year_control, nc = year_control_nc.split(".")
    
    
    model_decode_times_list = ["MIROC32","MPIESM12","MPIESM11"]
    if model in model_decode_times_list:
        tas_exp_global = xr.DataArray(xr.open_dataset(loc_tas+fname_exp,decode_times=False).data_vars["tas"]) #.loc[:,lat_mins[j]:lat_maxs[j],lon_mins[j]:lon_maxs[j]])
        tas_control_global = xr.DataArray(xr.open_dataset(loc_tas+fname_control,decode_times=False).data_vars["tas"]) #.loc[:,lat_mins[j]:lat_maxs[j],lon_mins[j]:lon_maxs[j]])
    else:
        tas_exp_global = xr.DataArray(xr.open_dataset(loc_tas+fname_exp).data_vars["tas"]) #.loc[:,lat_mins[j]:lat_maxs[j],lon_mins[j]:lon_maxs[j]])
        tas_control_global = xr.DataArray(xr.open_dataset(loc_tas+fname_control).data_vars["tas"]) #.loc[:,lat_mins[j]:lat_maxs[j],lon_mins[j]:lon_maxs[j]])
            
    if model == "HadCM3L":
        lat_tas = tas_exp_global.coords["latitude_1"]
        lon_tas = tas_exp_global.coords["longitude_1"]
    else:
        lat_tas = tas_exp_global.coords["lat"]
        lon_tas = tas_exp_global.coords["lon"]
    
    latshape = len(lat_tas)
    lonshape = len(lon_tas)
    
    
    # need to cosine-lat-weight the global mean
    wgt = np.zeros((latshape,lonshape))
    for ll in np.arange(0,lonshape,1):
        wgt[:,ll] = np.sqrt(np.cos(np.radians(lat_tas.values)))
    
    tas_baseline_global = tas_control_global.mean(dim="time")
    #gmst_baseline = np.average(tas_baseline_global.values[~np.isnan(tas_baseline_global.values)],weights=wgt[~np.isnan(tas_baseline_global.values)])
    indices = (~np.isnan(tas_baseline_global.values)) & (~np.isnan(wgt))
    gmst_baseline = np.average(tas_baseline_global.values[indices],weights=wgt[indices])
    
    tas_exp_np = tas_exp_global.values
    n_months_f = tas_exp_np.shape[0]
    time_exp = xr.cftime_range(start='0001', periods=n_months_f, freq='M')
    
    gmst = xr.DataArray(np.zeros(len(time_exp)),coords=[time_exp],dims=["time"])
    for tt in np.arange(0,len(time_exp),1):
        tas_tt = tas_exp_np[tt,:,:]
        #gmst[tt] = np.average(tas_tt[~np.isnan(tas_tt)],weights=wgt[~np.isnan(tas_tt)])    
        indices = (~np.isnan(tas_tt)) & (~np.isnan(wgt))
        gmst[tt] = np.average(tas_tt[indices],weights=wgt[indices])    
    
    gmst_anom = gmst - gmst_baseline
    gmst_anom_xr = xr.DataArray(gmst_anom.values,coords=[time_exp],dims=["time"])

    gmst_anom_xr.name = "gmst"
    gmst_anom_xr.attrs["creation_date"] = str(datetime.datetime.now())
    gmst_anom_xr.attrs["created_by"] = "Christopher Callahan, Christopher.W.Callahan.GR@dartmouth.edu"
    gmst_anom_xr.attrs["variable_description"] = "Global mean surface temperature anomaly"
    gmst_anom_xr.attrs["created_from"] = os.getcwd()+"/Calculate_GMST.ipynb"

    fname_out = loc_out+"GMST_anom_"+model+"_"+exp+".nc"
    gmst_anom_xr.to_netcdf(fname_out,mode="w")
    print(fname_out)